# D213 Task 1 Advanced Data Analytics

## Part 1

### A1: Research Question and Data Selection

Research Question:
Is it possible to at accurately determine customer sentiment from a customers review utilizing Natural language processing and nural networks?

Data and Rational:
The data that will be used for this analysis is "sentiment labeled sentences" dataset which can be found at the link bellow. This data set provides a sentence representing a review along with a label of 1 or 0 indicating a positive or negative sentiment respectively


https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences

### A2: Objectives

The objective of this analysis is to determine the feasibility of using a natural language processing neural network to determine a customers sentiment based on a review. The determination of feasibility will be made by creating a NLP model using Tensorflow. The objective of this model is to be able to take in the review text data and determine if the review has a positive or negative sentiment. 

### A3: Neural Network Type

# TODO

## Part 2

### B1: Data Exploration



In [66]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
# Extract data
amazon_df = pd.read_csv('./data/amazon_cells_labelled.txt', sep='\t', names=['review', 'score'])
imdb_df = pd.read_csv('./data/imdb_labelled.txt', sep='\t', names=['review', 'score'])
yelp_df = pd.read_csv('./data/yelp_labelled.txt', sep='\t', names=['review', 'score'])

print(f'Amazon Count: {amazon_df.shape[0]}')
print(f'IMDB Count: {imdb_df.shape[0]}')
print(f'Yelp Count: {yelp_df.shape[0]}')

# Label Data Source
amazon_df['source'] = 'amz'
imdb_df['source'] = 'imdb'
yelp_df['source'] = 'yelp'

# Join Dataframes 
df = pd.concat([amazon_df, imdb_df, yelp_df], ignore_index=True)
df.head()

Amazon Count: 1000
IMDB Count: 1000
Yelp Count: 1000


,review,score,source
0,So there is no way for me to plug it in here i...,0,amz
1,"Good case, Excellent value.",1,amz
2,Great for the jawbone.,1,amz
3,Tied to charger for conversations lasting more...,0,amz
4,The mic is great.,1,amz


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   score   3000 non-null   int64 
 2   source  3000 non-null   object
dtypes: int64(1), object(2)
memory usage: 70.4+ KB


In [73]:
# Vocabulary Size
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['review'])
vocab_size = len(vectorizer.get_feature_names_out())

print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 5284


In [72]:
# unique = set(df['review'].str.replace('[^a-zA-Z]', '').str.lower().str.split(' ').sum())
# unique

{'',
 'earbud',
 'snider',
 'fuzzy',
 'logic',
 'babysitting',
 'away',
 'apart',
 'thoughtprovoking',
 'main',
 'obliged',
 'dualpurpose',
 'tots',
 'true',
 'strong',
 'list',
 'caring',
 'informative',
 'believe',
 'summarize',
 'unless',
 'replacementr',
 'too',
 'option',
 'theres',
 'running',
 'them',
 'thus',
 'ngage',
 'vandiver',
 'wartime',
 'comparablypriced',
 'haggis',
 'nicolas',
 'hadnt',
 'mother',
 'adapter',
 'navigate',
 'designed',
 'combo',
 'sum',
 'unfortunate',
 'seated',
 'pleather',
 'senior',
 'gore',
 'brainsucking',
 'corny',
 'applifies',
 'crawfish',
 'shallow',
 'seuss',
 'customers',
 'including',
 'possibility',
 'arguing',
 'costcos',
 'intentions',
 'dishes',
 'livingworking',
 'ultracheap',
 'intended',
 'offers',
 'cardboard',
 'underbite',
 'headphones',
 'basically',
 'prelude',
 'warranty',
 'park',
 'exquisite',
 'appealing',
 'occasions',
 'essentially',
 'talent',
 'died',
 'when',
 'gives',
 'veggitarian',
 'comedic',
 'holds',
 'sack',
 'c

In [71]:
# Justification of max sequence length

df['review_length'] = df['review'].apply(lambda x: len(x.split()))
rl_mean = df['review_length'].mean()
rl_std = df['review_length'].std()
rl_max = df['review_length'].max()

print(f"Mean review length: {rl_mean}")
print(f"Standard deviation of review length: {rl_std}")
print(f"Maximum review length: {rl_max}")

# max length that covers around 95% of the dataset
cutoff_length = int(rl_mean + 2 * rl_std)
print(f"Suggested max sequence length: {cutoff_length}")


Mean review length: 11.704666666666666
Standard deviation of review length: 7.792839940633687
Maximum review length: 70
Suggested max sequence length: 27
